# MQTT credential CRUD
This module provides a comprehensive solution for managing MQTT credentials through Create, Retrieve, Update, and Delete (CRUD) operations. It leverages the `pycarta` library to interact with the backend via CartaAgent. Key functionalities include:

- **Authentication**: Establish secure communication using environment-based credentials.
- **Create**: Upload MQTT credentials from files, with options to overwrite or preserve existing data.
- **Retrieve**: List available credentials, fetch them as strings, or save them to files for local use.
- **Update**: Modify specific credential attributes while preserving others.
- **Delete**: Remove all credentials and validate successful deletion.

This module ensures secure, flexible, and automated credential lifecycle management, designed for use in IoT systems and broader CI/CD pipelines.

Below is a simple setup and workflow for CRUD operations for MQTT credentials in the testing environment.

In [1]:
import logging
import os
from pycarta import login

# Configure the root logger to WARNING or INFO
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s:%(name)s:%(message)s'
)
logger = logging.getLogger(__name__)

# 1. Authenticate with Carta. Use your own credentials here.
carta_root_username = os.getenv("CARTA_CHEN_USERNAME")
carta_root_password = os.getenv("CARTA_CHEN_PASSWORD")
base_url_dev = os.getenv("CARTA_HOST_DEV", "https://api.sandbox.carta.contextualize.us.com")
base_url_prod = os.getenv("CARTA_HOST_PROD", "https://api.carta.contextualize.us.com")
carta_host = base_url_dev  # Use base_url_dev for development environment, and use base_url_prod for production environment

if not all([carta_root_username, carta_root_password]):
    logger.error("CARTA_ROOT_USERNAME and CARTA_ROOT_PASSWORD environment variables must be set.")
else:
    login(username=carta_root_username, password=carta_root_password, host=carta_host)
    logger.info(f"Successfully authenticated with Carta host: {carta_host}.")

INFO:__main__:Successfully authenticated with Carta host: https://api.sandbox.carta.contextualize.us.com.


In [2]:
import pycarta.mqtt.credential as credential

# Define the directory path containing the credential files
credential_dir = "mb33-thing301"

# Define the file paths relative to the directory
file_paths = {
    "broker_address": "broker_address.pem",
    "certificate": "34c68a3ca20dfafa5a327c03529649d206b13a789c08df92be63bbea2514cb3d-certificate.pem.crt",
    "private_key": "34c68a3ca20dfafa5a327c03529649d206b13a789c08df92be63bbea2514cb3d-private.pem.key",
    "ca_certificate": "AmazonRootCA1.pem"
}

# Resolve full file paths using the directory path
resolved_file_paths = {key: os.path.join(credential_dir, value) for key, value in file_paths.items()}

# Set the tag for the credentials
tag = "mb33-thing301"

In [3]:
# 1. Upload the credentials
logger.info(f"Uploading MQTT credentials to tag: {tag}")
upload_success = credential.upload_mqtt_credentials(file_paths=resolved_file_paths, tag=tag)
logger.info(f"Upload success: {upload_success}")

INFO:__main__:Uploading MQTT credentials to tag: mb33-thing301
INFO:pycarta.mqtt.credential:Uploaded broker_address successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Uploaded certificate successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Uploaded private_key successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Uploaded ca_certificate successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Successfully uploaded all MQTT credentials to tag 'mb33-thing301'.
INFO:__main__:Upload success: True


In [4]:
# 1a. Upload MQTT credentials without overwrite
logger.info("\nUploading MQTT credentials without overwrite:")
upload_success_no_overwrite = credential.upload_mqtt_credentials(
    file_paths=resolved_file_paths,
    tag=tag,
    overwrite=False
)
if upload_success_no_overwrite:
    logger.info("Credentials uploaded successfully without overwrite.")
else:
    logger.info("Credentials upload skipped due to existing credentials.")

INFO:__main__:
Uploading MQTT credentials without overwrite:
INFO:__main__:Credentials upload skipped due to existing credentials.


In [5]:
# 1b. Upload MQTT credentials with overwrite
logger.info("\nUploading MQTT credentials with overwrite:")
upload_success_with_overwrite = credential.upload_mqtt_credentials(
    file_paths=resolved_file_paths,
    tag=tag,
    overwrite=True
)
if upload_success_with_overwrite:
    logger.info("Credentials uploaded successfully with overwrite.")
else:
    logger.info("Failed to upload credentials with overwrite.")

INFO:__main__:
Uploading MQTT credentials with overwrite:
INFO:pycarta.mqtt.credential:Updated broker_address successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Updated certificate successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Updated private_key successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Updated ca_certificate successfully to /mqtt_credentials/mb33-thing301.
INFO:pycarta.mqtt.credential:Successfully uploaded all MQTT credentials to tag 'mb33-thing301'.
INFO:__main__:Credentials uploaded successfully with overwrite.


In [6]:
# 2. List all available MQTT credentials
logger.info("\nListing all available MQTT credentials with details...")
available_credentials = credential.list_mqtt_credentials(tag=tag)
if available_credentials:
    logger.info("Available credentials:")
    for file_type, details in available_credentials.items():
        logger.info(f"{file_type}:")
        logger.info(f"  ID: {details['id']}")
        logger.info(f"  Name: {details['name']}")
        logger.info(f"  Path: {details['path']}\n")
else:
    logger.info("Failed to list available credentials.")

INFO:__main__:
Listing all available MQTT credentials with details...
INFO:pycarta.mqtt.credential:Available credentials for tag 'mb33-thing301': {'broker_address': {'id': '74ca1634-8f8a-098b-553b-fe6572e6977c', 'name': 'broker_address.pem', 'path': '/mqtt_credentials/mb33-thing301/broker_address.pem'}, 'certificate': {'id': '64ca1634-9017-3372-786c-3a4900df2393', 'name': 'certificate.pem', 'path': '/mqtt_credentials/mb33-thing301/certificate.pem'}, 'private_key': {'id': '90ca1634-909c-7c03-afa5-37a5ef54dd9a', 'name': 'private_key.pem', 'path': '/mqtt_credentials/mb33-thing301/private_key.pem'}, 'ca_certificate': {'id': '6cca1634-911b-68b5-48ac-72224d541b8a', 'name': 'ca_certificate.pem', 'path': '/mqtt_credentials/mb33-thing301/ca_certificate.pem'}}
INFO:pycarta.mqtt.credential:Available credentials for tag 'mb33-thing301': {'broker_address': {'id': '74ca1634-8f8a-098b-553b-fe6572e6977c', 'name': 'broker_address.pem', 'path': '/mqtt_credentials/mb33-thing301/broker_address.pem'}, 'cer

In [7]:
# 3. Retrieve MQTT credentials as strings for a specific tag
logger.info(f"\nRetrieving MQTT credentials as strings for tag: {tag}")
try:
    credentials_data = credential.retrieve_mqtt_credentials(tag=tag)
    if credentials_data:
        logger.info("Retrieved credentials:")
        for key, value in credentials_data.items():
            logger.info(f"{key}:\n***\n")  # Masking actual content
    else:
        logger.info(f"No credentials found for tag: {tag}")
except Exception as e:
    logger.error(f"Failed to retrieve credentials for tag {tag}: {e}")

INFO:__main__:
Retrieving MQTT credentials as strings for tag: mb33-thing301
INFO:pycarta.mqtt.credential:Downloaded broker_address successfully from tag 'mb33-thing301' (ID: 74ca1634-8f8a-098b-553b-fe6572e6977c).
INFO:pycarta.mqtt.credential:Downloaded certificate successfully from tag 'mb33-thing301' (ID: 64ca1634-9017-3372-786c-3a4900df2393).
INFO:pycarta.mqtt.credential:Downloaded private_key successfully from tag 'mb33-thing301' (ID: 90ca1634-909c-7c03-afa5-37a5ef54dd9a).
INFO:pycarta.mqtt.credential:Downloaded ca_certificate successfully from tag 'mb33-thing301' (ID: 6cca1634-911b-68b5-48ac-72224d541b8a).
INFO:pycarta.mqtt.credential:Successfully retrieved MQTT credentials for tag 'mb33-thing301'.
INFO:__main__:Retrieved credentials:
INFO:__main__:broker_address:
***

INFO:__main__:certificate:
***

INFO:__main__:private_key:
***

INFO:__main__:ca_certificate:
***



In [8]:
# 4. Retrieve MQTT credentials and save them to files for a specific tag
file_paths_retrieved = {
    "broker_address": "retrieved-credential/broker-address.pem",
    "certificate": "retrieved-credential/certificate.pem",
    "private_key": "retrieved-credential/private_key.pem",
    "ca_certificate": "retrieved-credential/ca_certificate.pem"
}

logger.info(f"\nRetrieving MQTT credentials for tag: {tag} and saving to files...")
try:
    credentials_data = credential.retrieve_mqtt_credentials(tag=tag, file_paths=file_paths_retrieved)
    if credentials_data:
        logger.info(f"Credentials successfully retrieved and saved to files for tag: {tag}")
    else:
        logger.info(f"Failed to retrieve or save credentials for tag: {tag}")
except Exception as e:
    logger.error(f"Error retrieving and saving credentials for tag {tag}: {e}")

INFO:__main__:
Retrieving MQTT credentials for tag: mb33-thing301 and saving to files...
INFO:pycarta.mqtt.credential:Downloaded broker_address successfully from tag 'mb33-thing301' (ID: 74ca1634-8f8a-098b-553b-fe6572e6977c).
INFO:pycarta.mqtt.credential:Saved broker_address to retrieved-credential/broker-address.pem for tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Downloaded certificate successfully from tag 'mb33-thing301' (ID: 64ca1634-9017-3372-786c-3a4900df2393).
INFO:pycarta.mqtt.credential:Saved certificate to retrieved-credential/certificate.pem for tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Downloaded private_key successfully from tag 'mb33-thing301' (ID: 90ca1634-909c-7c03-afa5-37a5ef54dd9a).
INFO:pycarta.mqtt.credential:Saved private_key to retrieved-credential/private_key.pem for tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Downloaded ca_certificate successfully from tag 'mb33-thing301' (ID: 6cca1634-911b-68b5-48ac-72224d541b8a).
INFO:pycarta.mqtt.credentia

In [9]:
# 5. Update specific MQTT credentials
update_data = {
    "broker_address": "newhost.example.com",
    "certificate": None,
    "private_key": None,
    "ca_certificate": None
}

logger.info(f"\nUpdating MQTT credentials for tag: {tag}")
try:
    update_success = credential.update_mqtt_credentials(updates=update_data, tag=tag)
    logger.info(f"Update success: {update_success}")

    # Verify updated credentials
    if update_success:
        logger.info(f"\nRetrieving updated MQTT credentials for tag: {tag}")
        updated_credentials = credential.retrieve_mqtt_credentials(tag=tag)
        if updated_credentials:
            logger.info("Updated credentials:")
            for key, value in updated_credentials.items():
                logger.info(f"{key}:\n***\n")
        else:
            logger.info(f"Failed to retrieve updated credentials for tag: {tag}")
    else:
        logger.info(f"Failed to update credentials for tag: {tag}")
except Exception as e:
    logger.error(f"Error updating credentials for tag {tag}: {e}")

INFO:__main__:
Updating MQTT credentials for tag: mb33-thing301
INFO:pycarta.mqtt.credential:Updated broker_address successfully to /mqtt_credentials/mb33-thing301.
INFO:__main__:Update success: True
INFO:__main__:
Retrieving updated MQTT credentials for tag: mb33-thing301
INFO:pycarta.mqtt.credential:Downloaded broker_address successfully from tag 'mb33-thing301' (ID: 74ca1634-8f8a-098b-553b-fe6572e6977c).
INFO:pycarta.mqtt.credential:Downloaded certificate successfully from tag 'mb33-thing301' (ID: 64ca1634-9017-3372-786c-3a4900df2393).
INFO:pycarta.mqtt.credential:Downloaded private_key successfully from tag 'mb33-thing301' (ID: 90ca1634-909c-7c03-afa5-37a5ef54dd9a).
INFO:pycarta.mqtt.credential:Downloaded ca_certificate successfully from tag 'mb33-thing301' (ID: 6cca1634-911b-68b5-48ac-72224d541b8a).
INFO:pycarta.mqtt.credential:Successfully retrieved MQTT credentials for tag 'mb33-thing301'.
INFO:__main__:Updated credentials:
INFO:__main__:broker_address:
***

INFO:__main__:certif

In [10]:
# 6. Delete all MQTT credentials
logger.info("\nDeleting all MQTT credentials...")
delete_success = credential.delete_mqtt_credentials(tag=tag)
logger.info(f"Delete success: {delete_success}")


INFO:__main__:
Deleting all MQTT credentials...
INFO:pycarta.mqtt.credential:Deleted broker_address successfully from tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Deleted certificate successfully from tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Deleted private_key successfully from tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Deleted ca_certificate successfully from tag 'mb33-thing301'.
INFO:pycarta.mqtt.credential:Successfully deleted all MQTT credentials for tag 'mb33-thing301'.
INFO:__main__:Delete success: True


In [11]:
# 7. Verify deletion
logger.info("\nVerifying deletion of MQTT credentials...")
deleted_credentials = credential.retrieve_mqtt_credentials(tag=tag)
if not deleted_credentials:
    logger.info("All credentials successfully deleted.")
else:
    logger.info(f"Failed to delete credentials. Remaining: {deleted_credentials}")

INFO:__main__:
Verifying deletion of MQTT credentials...
ERROR:pycarta.mqtt.credential:Failed to retrieve broker_address for tag 'mb33-thing301'.
INFO:__main__:All credentials successfully deleted.


In [22]:
# (Optional) Future usage example: create an authenticator agent
# from the retrieved credentials, if you want to pass them to an MQTT client
#
# from pycarta.mqtt.credential import BilateralCredentialAuthenticator
# if credentials_data:
#     agent = BilateralCredentialAuthenticator(
#         ca_cert=credentials_data[CertificateFileType.CA_CERTIFICATE.name.lower()],
#         key=credentials_data[CertificateFileType.PRIVATE_KEY.name.lower()],
#         cert=credentials_data[CertificateFileType.CERTIFICATE.name.lower()]
#     )
#     # then pass 'agent' to your publish/subscribe as needed
#     # e.g., my_publish(topic="...", agent=agent)
